In [6]:
%cd /content/

/content


In [7]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove

## downnload installation files
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz
path = "l_openvino_toolkit_p_2020.1.023.tgz"

CPU times: user 185 ms, sys: 31 ms, total: 216 ms
Wall time: 58.4 s


In [12]:
!tar xf "{path}"

In [21]:
!cd l_openvino_toolkit_p_2020.1.023/ && ./install_openvino_dependencies.sh && sed -i 's/decline/accept/g' silent.cfg && ./install.sh --silent silent.cfg


This script installs the following OpenVINO 3rd-party dependencies:
  1. GTK+, FFmpeg and GStreamer libraries used by OpenCV
  2. libusb library required for Myriad plugin for Inference Engine
  3. build dependencies for OpenVINO samples

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvi

In [24]:
!cd /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ && ls -al 

total 624
drwxr-xr-x 2 root root  4096 Jan 27  2020 .
drwxr-xr-x 7 root root  4096 Jan 27  2020 ..
-rw-r--r-- 1 root root  3818 Jan 24  2020 activation_ext.py
-rw-r--r-- 1 root root  1030 Jan 24  2020 argmax_ext.py
-rw-r--r-- 1 root root  2958 Jan 24  2020 assign_elimination.py
-rw-r--r-- 1 root root  8197 Jan 24  2020 basic_lstm_cell.py
-rw-r--r-- 1 root root  1404 Jan 24  2020 BatchMatMul_ext.py
-rw-r--r-- 1 root root  5061 Jan 24  2020 BatchToSpaceNDToUpsample.py
-rw-r--r-- 1 root root  1089 Jan 24  2020 BlockLSTM_ext.py
-rw-r--r-- 1 root root  6607 Jan 24  2020 BlockLSTM.py
-rw-r--r-- 1 root root  1046 Jan 24  2020 bucketize_ext.py
-rw-r--r-- 1 root root  1668 Jan 24  2020 bucketize.py
-rw-r--r-- 1 root root  1016 Jan 24  2020 Cast_ext.py
-rw-r--r-- 1 root root   937 Jan 24  2020 concat_ext.py
-rw-r--r-- 1 root root  1793 Jan 24  2020 concat.py
-rw-r--r-- 1 root root  1255 Jan 24  2020 const_ext.py
-rw-r--r-- 1 root root  4472 Jan 24  2020 conv_ext.py
-rw-r--r-- 1 root root  1388 J

In [33]:
!cp /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json /content/

In [34]:
!ls -la 

total 496332
drwxr-xr-x 1 root root      4096 Aug 18 02:21 .
drwxr-xr-x 1 root root      4096 Aug 18 02:12 ..
drwxr-xr-x 1 root root      4096 Aug 10 21:25 .config
drwxr-xr-x 4 root root      4096 Aug 18 02:12 l_openvino_toolkit_p_2020.1.023
-rw-r--r-- 1 root root 508213676 Jan 28  2020 l_openvino_toolkit_p_2020.1.023.tgz
drwxr-xr-x 1 root root      4096 Jul 30 16:30 sample_data
-rw-r--r-- 1 root root      1835 Aug 18 02:22 ssd_v2_support.json


In [35]:
#openvino fixes: edit 
# Read in the file, make sure the .json corresponds to the model!!!
with open('ssd_v2_support.json', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"Postprocessor/ToFloat"', '"Postprocessor/Cast_1"')

# Write the file out again
with open('ssd_v2_support.json', 'w') as file:
  file.write(filedata)

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [38]:
# Copy finetune model to local
!mkdir -p /content/fine_tune_model
!cp -rf /content/gdrive/My\ Drive/dataset/fine_tune_model/* /content/fine_tune_model/

In [39]:
!source /opt/intel/openvino/bin/setupvars.sh && \
python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model /content/fine_tune_model/fine_tune_model/frozen_inference_graph.pb \
--tensorflow_use_custom_operations_config ssd_v2_support.json \
--tensorflow_object_detection_api_pipeline_config /content/fine_tune_model/fine_tune_model/pipeline.config \
--reverse_input_channels \
--data_type FP16 \
--output_dir /content/gdrive/My\ Drive/dataset/

[setupvars.sh] OpenVINO environment initialized
[ WARNING ]  Use of deprecated cli option --tensorflow_use_custom_operations_config detected. Option use in the following releases will be fatal. Please use --transformations_config cli option instead
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/fine_tune_model/fine_tune_model/frozen_inference_graph.pb
	- Path for generated IR: 	/content/gdrive/My Drive/dataset/
	- IR output name: 	frozen_inference_graph
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input c

In [45]:
# OpenVino xml, bin to Blob file
blob_dir = "/content/gdrive/My Drive/dataset/openvino/"
xmlfile =  "/content/gdrive/My Drive/dataset/openvino/frozen_inference_graph.xml"
binfile =  "/content/gdrive/My Drive/dataset/openvino/frozen_inference_graph.bin"

In [41]:
import requests
#For openvino 20.01 use this link to compile the blobb
url = "http://69.164.214.171:8080"

In [42]:
payload = {'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4'}
files = [
  ('definition', open(xmlfile,'rb')),
  ('weights', open(binfile,'rb'))
]

In [43]:
response = requests.request("POST", url, data = payload, files = files)

In [46]:
blobnameraw = response.headers.get('Content-Disposition')
print(blobnameraw)
blobname = blobnameraw[blobnameraw.find('='):][1:]
with open(blob_dir + blobname, 'wb') as f:
  f.write(response.content)

attachment; filename=frozen_inference_graph.blob
